In [1]:
exec(open("./funcs/mmj_functions.py").read())
exec(open("./funcs/variant_of_Floyd_Warshall.py").read())
exec(open("./funcs/Prim_Minimum_Spanning_Tree.py").read())

In [2]:
import time

In [3]:
test_data_145 = pickle.load(  open( "./data/test_data_145.p", "rb" ) ) 
 
X = test_data_145[18]
    
# plt.scatter(X[:,0],X[:,1], s= 10)

In [4]:
def cal_n_mmj_new(X_plus_distance_matrix, X_plus_mmj_matrix, n):
    for i in range(n):
        X_plus_mmj_matrix[n,i] = X_plus_mmj_matrix[i,n] = mmj_n_to_r(X_plus_distance_matrix, X_plus_mmj_matrix, n, i)
 
    for i in range(n):        
        for j in range(n):
            if i < j:
                X_plus_mmj_matrix[i,j] = X_plus_mmj_matrix[j,i] = update_mmj_ij(X_plus_distance_matrix, X_plus_mmj_matrix, n, i, j)

def cal_mmj_matrix_algo_1_warm_start(X_plus, X_mmj_matrix):
    
    X_plus_distance_matrix = pairwise_distances(X_plus)
    X_plus_distance_matrix = np.round(X_plus_distance_matrix, 15)

    N = len(X_plus) - 1
   
    X_plus_mmj_matrix = np.zeros((N + 1,N + 1))
    
    for i in range(N):
        for j in range(N):
            X_plus_mmj_matrix[i,j] = X_mmj_matrix[i,j]
            
    cal_n_mmj_new(X_plus_distance_matrix, X_plus_mmj_matrix, N)
 
    return X_plus_mmj_matrix

In [5]:
X.shape

(500, 2)

In [6]:
new_pt =  np.random.random((1,2))
new_pt

array([[0.45842121, 0.76526635]])

In [7]:
# Check new point does not belong to X

for ii in X:
    if np.allclose(ii, new_pt) or np.sum(np.abs(ii - new_pt)) < 1e-10:
        print("new_pt is in X!")

In [8]:
X_plus = np.concatenate((X, new_pt), axis=0)
X_plus.shape

(501, 2)

In [9]:
# Test cold-start calculation of the APPD matrix by variant of the Floyd-Warshall algorithm.


start = time.time()
X_plus_mmj_matrix_Floyd_Warshall_python = cal_mmj_variant_of_Floyd_Warshall_python(X_plus)
end = time.time()
time_used = end - start
time_used = np.round(time_used, 3)
print(f"Cold start calculation of the APPD matrix by Floyd-Warshall algorithm, time_used: {time_used}s.")


Cold start calculation of the APPD matrix by Floyd-Warshall algorithm, time_used: 106.715s.


In [10]:
# Test cold-start calculation of the APPD matrix by MST_shortest_path algorithm.

start = time.time()
X_plus_mmj_matrix_MST_shortest_path = cal_mmj_matrix_by_shortest_path_on_minimum_spanning_tree(X_plus)
end = time.time()
time_used = end - start
time_used = np.round(time_used, 3)
print(f"Cold start calculation of the APPD matrix by MST_shortest_path algorithm, time_used: {time_used}s.")


Cold start calculation of the APPD matrix by MST_shortest_path algorithm, time_used: 25.984s.


In [11]:
# Test warm-start calculation of the APPD matrix by Algorithm 1.

X_mmj_matrix = cal_mmj_matrix_by_algo_4_Calculation_and_Copy(X)

start = time.time()
X_plus_mmj_matrix_algo_1_warm = cal_mmj_matrix_algo_1_warm_start(X_plus, X_mmj_matrix)
end = time.time()
time_used = end - start
time_used = np.round(time_used, 3)

print(f"Warm-start calculation of the APPD matrix by Algorithm 1, time_used: {time_used}s.")

Warm-start calculation of the APPD matrix by Algorithm 1, time_used: 3.153s.


In [12]:

print(np.allclose(X_plus_mmj_matrix_Floyd_Warshall_python, X_plus_mmj_matrix_algo_1_warm))
print(np.sum(np.abs(X_plus_mmj_matrix_Floyd_Warshall_python - X_plus_mmj_matrix_algo_1_warm)))


True
1.7459200751801518e-11


In [13]:
print(np.allclose(X_plus_mmj_matrix_MST_shortest_path, X_plus_mmj_matrix_algo_1_warm))
print(np.sum(np.abs(X_plus_mmj_matrix_MST_shortest_path - X_plus_mmj_matrix_algo_1_warm)))


True
2.137301446936135e-11


In [14]:
# Therefore, we can conclude warm-start of Algorithm 1 is faster than cold-start of other algorithms.
# This is especially useful when the graph is a directed dense graph, where we cannot use
# Algorithm 4 (MMJ distance by Calculation and Copy).